In [ ]:
from datasets import load_dataset

dataset = load_dataset('bookcorpus')

train_data = dataset['train'].shuffle(seed=42).select(range(1000)) 

bookcorpus.py:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

The repository for bookcorpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bookcorpus.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized_data = tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)
    tokenized_data['labels'] = tokenized_data['input_ids'].copy()
    return tokenized_data

train_data = train_data.map(tokenize_function, batched=True, remove_columns=['text'])
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')

model.config.pad_token_id = tokenizer.pad_token_id


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

# Define the LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  
    r=8,                           
    lora_alpha=32,                  
    lora_dropout=0.1,               
    target_modules=['c_attn', 'c_proj'],  
)

model = get_peft_model(model, lora_config)

# Freeze all non-LoRA parameters
for name, param in model.named_parameters():
    if 'lora' not in name:   
        param.requires_grad = False


In [ ]:

train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",         # Disable evaluation for now
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=200,
    save_steps=500,
    save_safetensors=False, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
)

trainer.train()

trainer.save_model('./fine-tuned-gpt2') 

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
200,0.086400


In [ ]:
model.save_pretrained('./fine-tuned-lora-gpt2')
tokenizer.save_pretrained('./fine-tuned-lora-gpt2')

('./fine-tuned-lora-gpt2/tokenizer_config.json',
 './fine-tuned-lora-gpt2/special_tokens_map.json',
 './fine-tuned-lora-gpt2/vocab.json',
 './fine-tuned-lora-gpt2/merges.txt',
 './fine-tuned-lora-gpt2/added_tokens.json')

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device) 
    outputs = model.generate(inputs['input_ids'], max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "Once upon a time in a forest"
generated_text = generate_text(prompt)
print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time in a forest, i had seen a strange figure in the darkness, and i knew he was a vampire.
